# SAPO Adaptive I/J: 5 GPT-2 Nodes with Gradient-Based Adaptation

This notebook runs an **ADAPTIVE** experiment where I/J ratio is dynamically optimized during training.

**Configuration:**
- **I+J=4** (total rollouts per round, varies dynamically)
- **Initial J=2** (starts at middle, adapts based on reward)
- **G=8** (completions per question)
- **Model**: GPT-2 (124M params)
- **Hardware**: 5 nodes (1 coordinator + 4 workers) on 1× A100 80GB
- **Algorithm**: Gradient-Based Adaptive I/J (Version 2 from ADAPTIVE_IJ_ALGORITHM.md)

**Purpose:** Demonstrate automated discovery of optimal I/J ratio through gradient ascent.

**Default Mode: TESTING (10 rounds, ~6 minutes)**
- Quick validation to ensure adaptive algorithm works
- See Cell 2 to switch to PRODUCTION mode (2000 rounds, ~21 hours)

**Expected Behavior:**
- J starts at 2 (I=2, J=2)
- Algorithm increases J if swarm sharing helps
- Algorithm decreases J if local training is better
- Should converge to optimal ratio (likely J=4 based on paper)

**Memory Usage:** ~30 GB peak VRAM (4 workers train, coordinator doesn't)

**Thesis Contribution:** Novel adaptive algorithm vs fixed I/J ratios in paper

**Run AFTER:** Baseline (EX12.14a) for comparison

## 1. Configuration

**This notebook uses ADAPTIVE I/J selection.**

**⚠️ IMPORTANT: Testing Mode Enabled by Default**
- Default: 10 rounds (~6 minutes) - validates adaptive algorithm
- Production: 2000 rounds (~21 hours) - full training with adaptation

**To switch to production mode:** Change `MAX_ROUNDS` in Cell 2 below.

In [ ]:
# SAPO Adaptive I/J Experiment Configuration
# Dynamically adjusts I/J ratio during training

# ============================================
# ADAPTIVE I/J CONFIGURATION
# ============================================
EXPERIMENT_NAME = 'sapo_gpt2_adaptive_ij'

# Total samples per round (I + J = 4)
TOTAL_SAMPLES = 4            # Total rollouts per round
INITIAL_J = 2                # Starting J value (middle of range)
INITIAL_I = TOTAL_SAMPLES - INITIAL_J  # Computed automatically

# Adaptive algorithm parameters
ADAPTIVE_IJ_ENABLED = True   # Enable adaptive I/J selection
ADAPTATION_RATE = 0.1        # Learning rate (alpha) for gradient updates
BASELINE_ALPHA = 0.95        # EMA smoothing for reward baseline

# ============================================
# TRAINING MODE: TESTING (default) or PRODUCTION
# ============================================
# TESTING MODE (default): Quick validation ~6 minutes
MAX_ROUNDS = 10              # Testing: 10 rounds to verify adaptive algorithm

# PRODUCTION MODE: Full training ~21 hours
# Uncomment line below for production run:
# MAX_ROUNDS = 2000          # Production: Full 2000 rounds for convergence

# ============================================
# FIXED SETTINGS (same for all experiments)
# ============================================
NUM_NODES = 5                # Run 5 nodes (1 coordinator + 4 workers)
MODEL_NAME = 'gpt2'          # GPT-2 (124M params, fits memory)
NUM_GENERATIONS = 8          # G: Completions per question (like paper)
SEED = 42                    # For reproducibility

# Rollout Sharing Configuration
ROLLOUT_PUBLISH_FREQUENCY = 'stage'  # When to share rollouts
ROLLOUT_CLEANUP_ENABLED = True       # Enable cleanup to save space
ROLLOUT_KEEP_LAST_N_ROUNDS = 20      # Keep recent rollouts only
ROLLOUT_ARCHIVE_OLD = False          # Don't archive (saves space)

# Checkpoint Configuration
CHECKPOINT_INTERVAL = 100    # Save checkpoints every 100 rounds
MAX_STAGES = 1               # Stages per round (1=default)

# Optional: HuggingFace Token
HUGGINGFACE_TOKEN = None  # Set to your token or keep None

# ============================================
# DISPLAY CONFIGURATION
# ============================================
mode = "TESTING" if MAX_ROUNDS <= 20 else "PRODUCTION"
estimated_time = "~6 minutes" if MAX_ROUNDS <= 20 else "~21 hours"

print("="*60)
print(f"SAPO Adaptive I/J Experiment - {mode} MODE")
print("="*60)
print(f"✓ Mode: {mode}")
print(f"✓ Nodes: {NUM_NODES} (1 coordinator + 4 workers on single A100 80GB)")
print(f"✓ Model: {MODEL_NAME}")
print(f"✓ Experiment: {EXPERIMENT_NAME}")
print(f"✓ Max Rounds: {MAX_ROUNDS}")
print()
print("🔄 ADAPTIVE I/J CONFIGURATION:")
print(f"   Total Samples: {TOTAL_SAMPLES} (I + J)")
print(f"   Initial: I={INITIAL_I}, J={INITIAL_J}")
print(f"   Adaptation Rate (α): {ADAPTATION_RATE}")
print(f"   Baseline Smoothing: {BASELINE_ALPHA}")
print()
print(f"Expected VRAM: ~30 GB peak (4 workers train, coordinator doesn't)")
print(f"Expected Time: {estimated_time}")
print()

if mode == "TESTING":
    print("🧪 TESTING MODE ENABLED")
    print("   Quick validation run - verifies:")
    print("   ✓ Adaptive algorithm updates I/J each round")
    print("   ✓ Logs show gradient updates")
    print("   ✓ System handles changing rollout counts")
    print()
    print("   After validation succeeds, uncomment production line")
    print("   in this cell to run full 2000-round training.")
else:
    print("📊 PRODUCTION MODE - Adaptive I/J")
    print("   Expected behavior:")
    print("   - J starts at 2 (middle of range)")
    print("   - Increases if swarm helps (likely converges to J=4)")
    print("   - Decreases if local training is better")
    print("   - Should match/exceed best fixed ratio (Config 2)")
    print()
    print("   ⚠️  This is a THESIS CONTRIBUTION - novel algorithm!")

print("="*60)

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set base path (MUST BE SAME ACROSS ALL NODES)
GDRIVE_BASE_PATH = '/content/drive/MyDrive/rl-swarm'
os.makedirs(GDRIVE_BASE_PATH, exist_ok=True)

print(f"✓ Google Drive mounted at: {GDRIVE_BASE_PATH}")

## 3. System Setup & GPU Verification

In [ ]:
import torch

print("="*60)
print("GPU Verification")
print("="*60)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"✓ GPU: {gpu_name}")
    print(f"✓ Total VRAM: {total_memory:.1f} GB")
    print()
    
    required_memory = (NUM_NODES - 1) * 6.5 + 4  # 4 workers + coordinator
    print(f"Memory Requirements:")
    print(f"  Workers: {NUM_NODES - 1} × 6.5 GB = {(NUM_NODES - 1) * 6.5:.1f} GB")
    print(f"  Coordinator: ~4 GB (non-training)")
    print(f"  Total estimated: {required_memory:.1f} GB")
    print(f"  Available: {total_memory:.1f} GB")
    print(f"  Margin: {total_memory - required_memory:.1f} GB")
    print()
    
    if total_memory < required_memory:
        raise RuntimeError(f"Insufficient GPU memory: need {required_memory:.0f} GB, have {total_memory:.1f} GB")
    else:
        print(f"✅ Sufficient VRAM for {NUM_NODES} nodes")
else:
    raise RuntimeError("No GPU detected! Select A100 GPU runtime: Runtime > Change runtime type > A100 GPU")

## 4. Clone Repository & Install Dependencies

In [ ]:
%cd /content

# Remove existing directory if it exists
if os.path.exists('/content/rl-swarm'):
    print("Removing existing repository...")
    !rm -rf /content/rl-swarm

# Clone fresh copy
print("Cloning repository...")
!git clone https://github.com/Elrashid/rl-swarm.git /content/rl-swarm

# Change to repo directory
%cd /content/rl-swarm

# Verify clone worked
if not os.path.exists('requirements.txt'):
    raise FileNotFoundError("Repository clone failed - requirements.txt not found")

print("✓ Repository cloned successfully")
print()

# Install dependencies
print("Installing dependencies (this may take 3-5 minutes)...")
!pip install -q -r requirements.txt
!pip install -q 'protobuf>=4.25.0,<5.0'

# Verify installation
try:
    import reasoning_gym
    from rgym_exp.src.adaptive_ij import GradientAdaptiveIJ
    print()
    print("✓ Dependencies installed successfully")
    print("✓ reasoning-gym verified")
    print("✓ Adaptive I/J algorithm verified")
except ImportError as e:
    print()
    print("❌ ERROR: Installation failed!")
    print(f"   {e}")
    raise

## 5. Initialize Experiment

In [ ]:
from rgym_exp.utils.experiment_manager import init_experiment

# Initialize experiment structure in Google Drive
config_overrides = {
    'training.max_round': MAX_ROUNDS,
    'training.num_generations': NUM_GENERATIONS,
    'training.total_samples': TOTAL_SAMPLES,
    'training.initial_I': INITIAL_I,
    'training.initial_J': INITIAL_J,
    'training.adaptive_ij_enabled': ADAPTIVE_IJ_ENABLED,
    'training.adaptation_rate': ADAPTATION_RATE,
    'training.seed': SEED,
}

init_experiment(
    gdrive_base_path=GDRIVE_BASE_PATH,
    experiment_name=EXPERIMENT_NAME,
    config_overrides=config_overrides
)

print(f"✓ Experiment initialized: {EXPERIMENT_NAME}")
print(f"  Path: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}")
print(f"  Config: Initial I={INITIAL_I}, J={INITIAL_J} (total={TOTAL_SAMPLES})")
print(f"  Adaptive: Enabled with α={ADAPTATION_RATE}")
print()
print("🔄 I/J will adapt dynamically based on reward feedback!")

## 6. Launch 5-Node Adaptive Swarm (KEY CELL)

**This cell:**
1. Spawns 5 separate Python processes
2. Each process runs `swarm_launcher.py` with unique NODE_ID
3. All processes share GPU 0 (CUDA_VISIBLE_DEVICES=0)
4. Coordinator (node_0) manages round progression
5. Workers (node_1-4) follow coordinator

**Note for Adaptive:** All nodes use adaptive I/J algorithm - J will change each round based on performance. The algorithm automatically adjusts the ratio of local to external rollouts to maximize reward.

**Logs:** Each node writes to Google Drive at `{GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}/logs/`

**Monitor:** Use next cell (Cell 7) to track progress in real-time

In [ ]:
import subprocess
import os
import time
from datetime import datetime

mode_label = "TESTING" if MAX_ROUNDS <= 20 else "PRODUCTION"
estimated_duration = "~6 minutes" if MAX_ROUNDS <= 20 else "~21 hours"

print("="*60)
print(f"Launching {NUM_NODES}-Node Adaptive Swarm ({mode_label} MODE)")
print("="*60)
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Model: {MODEL_NAME}")
print(f"Initial Config: I={INITIAL_I}, J={INITIAL_J}, G={NUM_GENERATIONS}")
print(f"Adaptation: α={ADAPTATION_RATE} (will adjust I/J each round)")
print(f"Rounds: {MAX_ROUNDS} ({estimated_duration})")
print(f"Hardware: All {NUM_NODES} nodes on single GPU (A100 80GB)")
print("="*60)
print()

# =========================================
# CODE VERSION CHECK
# =========================================
print("Checking code version...")
result = subprocess.run(['git', 'log', '--oneline', '-5'],
                       capture_output=True, text=True, cwd='/content/rl-swarm')
commits = result.stdout.strip().split('\n')
latest_commit = commits[0] if commits else "unknown"

print(f"Latest commit: {latest_commit}")
print(f"Recent commits:")
for commit in commits[:3]:
    print(f"  {commit}")
print()

# Check for rollout publishing fix (critical for adaptive - needs rollout sharing!)
has_rollout_fix = any('027fb2d' in commit or 'rollout publishing' in commit.lower()
                      for commit in commits)

if has_rollout_fix:
    print("✓ Has rollout publishing fix (commit 027fb2d)")
else:
    print("⚠️  WARNING: Missing rollout publishing fix!")
    print("   Adaptive algorithm REQUIRES rollout sharing to work!")
    print("   Expected: commit 027fb2d 'fix: Implement proper rollout publishing'")

print()
print("="*60)
print("Starting node processes...")
print("="*60)
print()

processes = []
process_stderr = []  # Store stderr for each process
start_time = time.time()  # For ETA calculation

for node_id in range(NUM_NODES):
    # Environment variables for this node
    env = os.environ.copy()
    env['NODE_ID'] = f'node_{node_id}'
    env['NODE_ROLE'] = 'coordinator' if node_id == 0 else 'worker'
    env['MODEL_NAME'] = MODEL_NAME
    env['NUM_TRAIN_SAMPLES'] = str(INITIAL_I)  # Starting I value
    env['NUM_TRANSPLANT_TREES'] = str(INITIAL_J)  # Starting J value
    env['NUM_GENERATIONS'] = str(NUM_GENERATIONS)
    env['MAX_ROUNDS'] = str(MAX_ROUNDS)
    env['EXPERIMENT_NAME'] = EXPERIMENT_NAME
    env['GDRIVE_PATH'] = GDRIVE_BASE_PATH
    env['CUDA_VISIBLE_DEVICES'] = '0'  # All nodes share GPU 0
    env['SEED'] = str(SEED + node_id)  # Different seed per node (diversity)

    # Adaptive I/J configuration
    env['ADAPTIVE_IJ_ENABLED'] = 'True'
    env['ADAPTIVE_IJ_ALPHA'] = str(ADAPTATION_RATE)
    env['ADAPTIVE_IJ_BASELINE_ALPHA'] = str(BASELINE_ALPHA)
    env['ADAPTIVE_IJ_INITIAL_J'] = str(INITIAL_J)

    # Rollout sharing configuration
    env['ROLLOUT_PUBLISH_FREQUENCY'] = ROLLOUT_PUBLISH_FREQUENCY
    env['ROLLOUT_CLEANUP_ENABLED'] = str(ROLLOUT_CLEANUP_ENABLED)
    env['ROLLOUT_KEEP_LAST_N_ROUNDS'] = str(ROLLOUT_KEEP_LAST_N_ROUNDS)
    env['ROLLOUT_ARCHIVE_OLD'] = str(ROLLOUT_ARCHIVE_OLD)
    env['CHECKPOINT_INTERVAL'] = str(CHECKPOINT_INTERVAL)
    env['MAX_STAGES'] = str(MAX_STAGES)

    # Enable debug logging for state.trees tracking
    env['DEBUG_TREES'] = 'true'

    if HUGGINGFACE_TOKEN:
        env['HUGGINGFACE_ACCESS_TOKEN'] = HUGGINGFACE_TOKEN

    # Launch process with stderr/stdout capture
    import sys
    process = subprocess.Popen(
        [sys.executable, '-m', 'rgym_exp.runner.swarm_launcher'],
        env=env,
        cwd='/content/rl-swarm',
        stderr=subprocess.PIPE,  # Capture errors
        stdout=subprocess.PIPE,  # Capture output
        text=True
    )
    processes.append(process)
    process_stderr.append(process.stderr)

    role = "COORDINATOR" if node_id == 0 else "WORKER     "
    print(f"✓ Started node_{node_id} ({role}) - PID: {process.pid:5d}")

    # Stagger startup: coordinator gets 10s, workers get 5s
    delay = 10 if node_id == 0 else 5
    time.sleep(delay)

print()
print("="*60)
print("PROCESS HEALTH CHECK")
print("="*60)
print("Waiting 30 seconds for processes to initialize...")
time.sleep(30)

crashed_nodes = []
for i, p in enumerate(processes):
    returncode = p.poll()
    if returncode is not None:
        crashed_nodes.append(i)
        print(f"❌ node_{i}: CRASHED (exit code {returncode})")
        # Try to read stderr
        try:
            stderr_output = process_stderr[i].read()
            if stderr_output:
                print(f"   Error output:")
                for line in stderr_output.split('\n')[-10:]:  # Last 10 lines
                    if line.strip():
                        print(f"     {line}")
        except:
            print(f"   (Could not read error output)")
    else:
        print(f"✓ node_{i}: RUNNING")

print()

if crashed_nodes:
    print(f"⚠️  WARNING: {len(crashed_nodes)}/{NUM_NODES} nodes crashed!")
    print(f"   Crashed: {', '.join(f'node_{i}' for i in crashed_nodes)}")
    print(f"   Check logs in: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}/logs/")
    print()
    user_input = input("Continue with remaining nodes? (yes/no): ")
    if user_input.lower() != 'yes':
        print("Terminating all processes...")
        for p in processes:
            if p.poll() is None:
                p.terminate()
        raise RuntimeError(f"{len(crashed_nodes)} nodes crashed - see errors above")
else:
    print(f"✅ All {NUM_NODES} nodes launched successfully!")

print()
print(f"✓ Training will run for {estimated_duration} ({MAX_ROUNDS} rounds)")
print(f"✓ I/J will adapt each round based on reward")
print(f"✓ Logs location: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}/logs/")
print(f"✓ Checkpoints: Every {CHECKPOINT_INTERVAL} rounds")
print()
if MAX_ROUNDS <= 20:
    print("🧪 TESTING MODE: After completion, check Cell 8 for results")
    print("   Then uncomment production line in Cell 2 for full training")
else:
    print("⚠️  Keep this notebook open (browser tab active)")
    print("⚠️  Colab may disconnect after 12-24 hours")
    print("⚠️  Training will continue - use Cell 7.5 to check progress after reconnect")
print()
print("Monitor adaptive behavior in Cell 7 below...")

## 7. Monitor Adaptive Training Progress

**Watch how I/J adapts in real-time!**

In [ ]:
import time
from IPython.display import clear_output
import pandas as pd

print("Starting training monitor...")
print("Press 'Stop' button or Ctrl+C to interrupt\n")

monitor_start_time = time.time()

try:
    while True:
        clear_output(wait=True)

        # Check process status
        running = sum(1 for p in processes if p.poll() is None)
        completed = NUM_NODES - running

        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        elapsed_hours = (time.time() - start_time) / 3600

        print("="*70)
        print(f" SAPO Training Monitor - {EXPERIMENT_NAME}")
        print(f" Time: {current_time} | Elapsed: {elapsed_hours:.1f}h")
        print("="*70)
        print()

        # Node status
        print(f"Nodes: {running}/{NUM_NODES} running, {completed} completed")
        print()

        # GPU memory
        if torch.cuda.is_available():
            reserved = torch.cuda.memory_reserved(0) / 1e9
            total = torch.cuda.get_device_properties(0).total_memory / 1e9
            utilization = (reserved / total) * 100

            print(f"GPU: {reserved:.1f} / {total:.1f} GB ({utilization:.1f}%)")

            if utilization > 90:
                print(f"  ⚠️  WARNING: High memory usage!")
            elif utilization > 75:
                print(f"  ⚠️  Memory usage elevated")
            print()

        # Training progress
        try:
            from rgym_exp.utils.experiment_manager import get_experiment_status
            status = get_experiment_status(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

            if status:
                current_round = status.get('current_round', 0)
                progress_pct = (current_round / MAX_ROUNDS) * 100

                print(f"Progress:")
                print(f"  Round: {current_round:4d} / {MAX_ROUNDS} ({progress_pct:5.1f}%)")

                # Progress bar
                bar_length = 50
                filled = int(bar_length * progress_pct / 100)
                bar = '█' * filled + '░' * (bar_length - filled)
                print(f"  [{bar}]")

                # ETA
                if current_round > 10:
                    hours_per_round = elapsed_hours / current_round
                    remaining_rounds = MAX_ROUNDS - current_round
                    eta_hours = remaining_rounds * hours_per_round
                    print(f"  ETA: {eta_hours:.1f} hours (~{eta_hours/24:.1f} days)")

                print()

                # ==========================================
                # TOTAL CUMULATIVE REWARD (PRIMARY METRIC)
                # ==========================================
                if 'total_cumulative_reward' in status:
                    cumulative_rewards = status.get('cumulative_rewards', {})
                    total_cumulative = status['total_cumulative_reward']

                    print("┌" + "─"*68 + "┐")
                    print("│" + " TOTAL CUMULATIVE REWARD (SAPO Paper Metric)".center(68) + "│")
                    print("├" + "─"*68 + "┤")
                    print(f"│  TOTAL: {total_cumulative:8.2f}".ljust(69) + "│")

                    if cumulative_rewards:
                        print("│".ljust(69) + "│")
                        print("│  Per-Node Breakdown:".ljust(69) + "│")
                        for node_id in sorted(cumulative_rewards.keys()):
                            node_reward = cumulative_rewards[node_id]
                            pct = (node_reward / total_cumulative * 100) if total_cumulative > 0 else 0
                            line = f"│    {node_id:10s}: {node_reward:7.2f} ({pct:5.1f}%)"
                            print(line.ljust(69) + "│")

                    # Average per round
                    avg_per_round = total_cumulative / current_round if current_round > 0 else 0
                    print("│".ljust(69) + "│")
                    print(f"│  Avg per round: {avg_per_round:6.4f}".ljust(69) + "│")

                    # Comparison to paper benchmarks
                    print("│".ljust(69) + "│")
                    print("│  Paper Benchmarks (Qwen2.5, 2000 rounds):".ljust(69) + "│")
                    print("│    Baseline (8/0):    562".ljust(69) + "│")
                    print("│    Config 2 (4/4):  1,093 (+94%) ⭐".ljust(69) + "│")
                    print("└" + "─"*68 + "┘")
                    print()

                # Recent performance
                try:
                    from rgym_exp.utils.experiment_manager import get_experiment_metrics
                    df = get_experiment_metrics(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

                    if not df.empty:
                        recent_reward = df.tail(10)['my_reward'].mean()
                        print(f"Recent Performance:")
                        print(f"  Avg reward: {recent_reward:6.4f} (last 10 rounds)")
                        print()
                except Exception:
                    pass

        except Exception as e:
            print(f"Progress: Unable to load status ({e})")
            print()

        # Instructions
        print("-"*70)
        print("Press 'Stop' button or Ctrl+C to halt training")
        print(f"Next update in 60 seconds...")

        # Exit if all completed
        if running == 0:
            print()
            print("="*70)
            print("✅ All nodes completed successfully!")
            print("="*70)
            break

        time.sleep(60)

except KeyboardInterrupt:
    print("\n" + "="*70)
    print("⚠️  Training interrupted by user")
    print("="*70)
    print("\nTerminating all node processes...")

    for i, p in enumerate(processes):
        if p.poll() is None:
            print(f"  Stopping node_{i}... (PID: {p.pid})")
            p.terminate()

    time.sleep(5)

    for i, p in enumerate(processes):
        if p.poll() is None:
            print(f"  Force killing node_{i}... (PID: {p.pid})")
            p.kill()

    print("\n✓ All processes terminated")
    print("\n💾 Note: Training state is checkpointed.")
    print("   Re-run this notebook to resume from last checkpoint.")

## 7.5. Check Real-Time Progress from GDrive (Optional)

**Reconnected after disconnect?** Run this cell to check training progress:
- Shows current round for each node
- Displays elapsed time and adaptive I/J values
- Works even if your notebook disconnected

Progress is saved to GDrive every round, logs flush every 30 seconds.

In [ ]:
# === Real-Time Progress Viewer ===
# Run this cell anytime to check progress from GDrive
# Useful if you reconnect after notebook disconnect

import sys
sys.path.append('/content/rl-swarm')

from rgym_exp.utils.experiment_manager import get_experiment_status
from rgym_exp.utils.progress_tracker import get_experiment_progress

# Get comprehensive status including cumulative rewards
status = get_experiment_status(GDRIVE_BASE_PATH, EXPERIMENT_NAME)
progress = get_experiment_progress(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

print("="*70)
print("REAL-TIME PROGRESS FROM GDRIVE (ADAPTIVE I/J)")
print("="*70)
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Round: {status.get('current_round', 0)} / {MAX_ROUNDS}")
print(f"Active peers: {status.get('active_peers', 0)}")
print()

# Cumulative Rewards (SAPO paper metric)
if 'total_cumulative_reward' in status:
    cumulative_rewards = status.get('cumulative_rewards', {})
    total_cumulative = status['total_cumulative_reward']
    
    print("Cumulative Rewards (ADAPTIVE - SAPO paper metric):")
    print(f"  Total (all nodes): {total_cumulative:7.2f}")
    if cumulative_rewards:
        print(f"  Per node:")
        for node_id in sorted(cumulative_rewards.keys()):
            print(f"    {node_id:10s}: {cumulative_rewards[node_id]:7.2f}")
    print()
else:
    print("Cumulative rewards: Not available yet (training just started)")
    print()

# Per-node progress details
print("Per-Node Progress:")
for node_id, node_data in progress.get('nodes', {}).items():
    if 'error' in node_data:
        print(f"  {node_id}: {node_data['error']}")
    else:
        latest_round = node_data.get('latest_round', 'N/A')
        elapsed_sec = node_data.get('elapsed_seconds', 0)
        elapsed_hours = elapsed_sec / 3600
        print(f"  {node_id}: Round {latest_round} | {elapsed_hours:.1f}h elapsed")

print()
print("="*70)
print("Note: Progress updates every round. Logs flush every 30s to GDrive.")
print("Watch adaptive I/J values evolve in Cell 8 analysis!")

## 8. Analyze Adaptive Behavior

**See how I/J evolved during training!**

In [ ]:
from rgym_exp.utils.experiment_manager import get_experiment_metrics
import matplotlib.pyplot as plt
import numpy as np

print("="*70)
print(f"Adaptive I/J Results: {EXPERIMENT_NAME}")
print("="*70)
print(f"Model: {MODEL_NAME}")
print(f"Rounds: {MAX_ROUNDS}")
print(f"Initial: I={INITIAL_I}, J={INITIAL_J}")
print(f"Adaptation Rate: α={ADAPTATION_RATE}")
print()

df = get_experiment_metrics(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

if not df.empty and 'adaptive_J' in df.columns:
    # Aggregate by round (take last value per round)
    round_df = df.groupby('round').last().reset_index()
    
    final_I = int(round_df['adaptive_I'].iloc[-1])
    final_J = int(round_df['adaptive_J'].iloc[-1])
    final_J_cont = round_df['adaptive_J_continuous'].iloc[-1]
    
    total_reward = df['my_reward'].sum()
    
    print("Final Adaptive State:")
    print(f"  I={final_I}, J={final_J} (J_continuous={final_J_cont:.2f})")
    print(f"  Total Reward: {total_reward:.2f}")
    print()
    
    # Statistics
    mean_J = round_df['adaptive_J'].mean()
    std_J = round_df['adaptive_J'].std()
    
    print("Adaptive Statistics:")
    print(f"  Mean J: {mean_J:.2f} ± {std_J:.2f}")
    print(f"  Started at: J={INITIAL_J}")
    print(f"  Ended at: J={final_J}")
    print(f"  Change: {final_J - INITIAL_J:+d} ({(final_J - INITIAL_J) / INITIAL_J * 100:+.0f}%)")
    print()
    
    # Compare to baseline (if available)
    print("Comparison to Fixed Ratios:")
    print("  (Based on SAPO paper expectations)")
    print(f"  Config 1 (I=3, J=1): +52% improvement")
    print(f"  Config 2 (I=2, J=2): +94% improvement (BEST fixed)")
    print(f"  Config 3 (I=1, J=3): +68% improvement")
    print(f"  Adaptive (I={final_I}, J={final_J}): TBD (run baseline first)")
    print()
    
    # Plot adaptive trajectory
    if MAX_ROUNDS >= 20:
        fig, axes = plt.subplots(2, 1, figsize=(12, 10))
        
        # Plot 1: J evolution over time
        ax1 = axes[0]
        ax1.plot(round_df['round'], round_df['adaptive_J_continuous'], 
                label='J (continuous)', linewidth=2, alpha=0.7)
        ax1.plot(round_df['round'], round_df['adaptive_J'], 
                label='J (discrete)', marker='o', markersize=3, alpha=0.5)
        ax1.axhline(y=INITIAL_J, color='gray', linestyle='--', 
                   label=f'Initial J={INITIAL_J}')
        ax1.set_xlabel('Round')
        ax1.set_ylabel('J (External Rollouts)')
        ax1.set_title('Adaptive I/J Evolution Over Training', fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Cumulative rewards
        ax2 = axes[1]
        for node_id in df['node_id'].unique():
            node_df = df[df['node_id'] == node_id].sort_values('round')
            ax2.plot(node_df['round'], node_df['my_reward'].cumsum(), 
                    label=node_id, alpha=0.7)
        ax2.set_xlabel('Round')
        ax2.set_ylabel('Cumulative Reward')
        ax2.set_title('Cumulative Rewards with Adaptive I/J', fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plot_path = f'{GDRIVE_BASE_PATH}/adaptive_ij_results.png'
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        print(f"✓ Plot saved: {plot_path}")
        plt.show()
        
        # Interpretation
        print()
        print("="*70)
        print("INTERPRETATION:")
        print("="*70)
        if final_J > INITIAL_J + 0.5:
            print("✅ Algorithm increased J → Swarm sharing is beneficial!")
            print("   External rollouts help more than local ones.")
        elif final_J < INITIAL_J - 0.5:
            print("⚠️  Algorithm decreased J → Local training preferred")
            print("   More benefit from own rollouts than swarm.")
        else:
            print("➡️  Algorithm kept J near initial → Balanced split optimal")
            print("   Local and external rollouts equally valuable.")
        print()
        
        if final_J >= 2:
            print("📊 THESIS INSIGHT:")
            print("   Adaptive algorithm discovered swarm collaboration value!")
            print("   This validates the need for external rollouts (J > 0).")
        else:
            print("📊 THESIS INSIGHT:")
            print("   Adaptive algorithm favored local training.")
            print("   May indicate: model too weak, swarm too small, or")
            print("   early training phase benefits from focused exploration.")
else:
    print("❌ No adaptive metrics found - verify ADAPTIVE_IJ_ENABLED=True")